In [ ]:
# ============================================================
# Run SQL Server query locally using Windows Authentication
# ============================================================

import pyodbc
import pandas as pd

# --- PARAMETERS ---
SERVER = r"QWVIDBSQLB401.ngquotit.com"      # your SQL Server or instance
DATABASE = "NGCS"                           # target database
DRIVER = "ODBC Driver 17 for SQL Server"    # or "ODBC Driver 18 for SQL Server"
LOAD_TASK_ID = 13449
COMPANY_ISSUER_ID = 2204
OUT_CSV = "mapping_results.csv"

# --- CONNECTION ---
conn_str = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    "Trusted_Connection=yes;"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

print("Connecting with connection string:")
print(conn_str)

conn = pyodbc.connect(conn_str)
print("Connected successfully ✅")

# --- SQL QUERY (no DECLARE or SET NOCOUNT) ---
sql = f"""
SELECT
    p.IssuerPlanId,
    s.PolicyNumber,
    cpp.ProductName
FROM dbo.STGPlanMappingFactors s (NOLOCK)
OUTER APPLY (
    SELECT TOP 1 p.*
    FROM dbo.PlanMappingIssuerDetail p (NOLOCK)
    LEFT JOIN dbo.PlanMappingIssuerDetailState ps (NOLOCK)
        ON p.PlanMappingIssuerDetailId = ps.PlanMappingIssuerDetailId
    WHERE
        (p.IssuerPlanName = s.PlanName OR p.IssuerPlanName IS NULL OR p.IssuerPlanName = '')
        AND (p.IssuerPlanId = s.PlanId OR p.IssuerPlanId IS NULL OR p.IssuerPlanId = '')
        AND (p.[Year] = s.[Year] OR p.[Year] IS NULL)
        AND (p.PercentRate = s.RatePercent OR p.PercentRate IS NULL)
        AND (p.CurrencyRate = s.RateDollar OR p.CurrencyRate IS NULL)
        AND (ps.StateCode = s.StateCode OR ps.StateCode IS NULL)
    ORDER BY
        CASE WHEN p.IssuerPlanName = s.PlanName THEN 1 ELSE 0 END +
        CASE WHEN p.IssuerPlanId   = s.PlanId   THEN 1 ELSE 0 END +
        CASE WHEN p.PercentRate    = s.RatePercent THEN 1 ELSE 0 END +
        CASE WHEN p.[Year]         = s.[Year]   THEN 1 ELSE 0 END +
        CASE WHEN p.CurrencyRate   = s.RateDollar THEN 1 ELSE 0 END +
        CASE WHEN ps.StateCode     = s.StateCode THEN 1 ELSE 0 END
        DESC
) p
JOIN dbo.CompanyProduct cp WITH (NOLOCK)
    ON cp.Deleted = 0
   AND cp.CompanyIssuerId = {COMPANY_ISSUER_ID}
JOIN dbo.CompanyProductPlan cpp WITH (NOLOCK)
    ON cpp.Deleted = 0
   AND cpp.CompanyProductId = cp.CompanyProductId
   AND p.CompanyProductPlanId = cpp.CompanyProductPlanId
WHERE s.LoadTaskId = {LOAD_TASK_ID};
"""

# --- EXECUTE AND FETCH ---
df = pd.read_sql(sql, conn)

print(f"✅ Query executed successfully. Rows returned: {len(df)}")
display(df.head(10))

# --- SAVE TO CSV ---
df.to_csv(OUT_CSV, index=False)
print(f"Results saved to: {OUT_CSV}")

conn.close()
print("Connection closed.")


In [ ]:
# ======================================================
# Cell 1: manhattan_mapping.py (kept separate for clarity)
# ======================================================

import pandas as pd
import pyodbc
import os

def is_manhattan_issuer(name: str) -> bool:
    """Return True if issuer name looks like 'Manhattan Life' (case/space tolerant)."""
    if not name:
        return False
    key = "".join(name.lower().split())
    return key in {"manhattanlife", "manhattan_life", "manhattan-life"}


def apply_manhattan_mapping(
    out_df: pd.DataFrame,
    raw_df: pd.DataFrame,
    plan_code_col: str = "PlanCode",
    *,
    load_task_id: int,
    company_issuer_id: int,
    server: str,
    database: str,
    driver: str = "ODBC Driver 18 for SQL Server",
    encrypt: bool = False,
    trust_server_certificate: bool = True,
) -> pd.DataFrame:
    """
    Fetch SQL mapping and apply ProductType <- PolicyNumber, PlanName <- ProductName.
    """
    if plan_code_col not in raw_df.columns:
        print("[ManhattanLife] PlanCode column missing — skipping mapping.")
        return out_df

    conn_str = (
        f"DRIVER={{{driver}}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        "Trusted_Connection=yes;"
        f"Encrypt={'yes' if encrypt else 'no'};"
        f"TrustServerCertificate={'yes' if trust_server_certificate else 'no'};"
    )

    print(f"[DB] Connecting to {server} | DB={database}")
    with pyodbc.connect(conn_str) as conn:
        sql = r"""
        SELECT DISTINCT
            p.IssuerPlanId   AS PlanCode,
            s.PolicyNumber,
            cpp.ProductName
        FROM dbo.STGPlanMappingFactors s (NOLOCK)
        OUTER APPLY (
            SELECT TOP 1 p.*
            FROM dbo.PlanMappingIssuerDetail p (NOLOCK)
            LEFT JOIN dbo.PlanMappingIssuerDetailState ps (NOLOCK)
                ON p.PlanMappingIssuerDetailId = ps.PlanMappingIssuerDetailId
            WHERE
                (p.IssuerPlanName = s.PlanName OR p.IssuerPlanName IS NULL OR p.IssuerPlanName = '')
                AND (p.IssuerPlanId = s.PlanId OR p.IssuerPlanId IS NULL OR p.IssuerPlanId = '')
                AND (p.[Year] = s.[Year] OR p.[Year] IS NULL)
                AND (p.PercentRate = s.RatePercent OR p.PercentRate IS NULL)
                AND (p.CurrencyRate = s.RateDollar OR p.CurrencyRate IS NULL)
                AND (ps.StateCode = s.StateCode OR ps.StateCode IS NULL)
            ORDER BY
                CASE WHEN p.IssuerPlanName = s.PlanName THEN 1 ELSE 0 END +
                CASE WHEN p.IssuerPlanId   = s.PlanId   THEN 1 ELSE 0 END +
                CASE WHEN p.PercentRate    = s.RatePercent THEN 1 ELSE 0 END +
                CASE WHEN p.[Year]         = s.[Year]   THEN 1 ELSE 0 END +
                CASE WHEN p.CurrencyRate   = s.RateDollar THEN 1 ELSE 0 END +
                CASE WHEN ps.StateCode     = s.StateCode THEN 1 ELSE 0 END
                DESC
        ) p
        JOIN dbo.CompanyProduct cp WITH (NOLOCK)
            ON cp.Deleted = 0
           AND cp.CompanyIssuerId = ?
        JOIN dbo.CompanyProductPlan cpp WITH (NOLOCK)
            ON cpp.Deleted = 0
           AND cpp.CompanyProductId = cp.CompanyProductId
           AND p.CompanyProductPlanId = cpp.CompanyProductPlanId
        WHERE s.LoadTaskId = ?;
        """
        map_df = pd.read_sql(sql, conn, params=[company_issuer_id, load_task_id]).fillna("")
        if map_df.empty:
            print("[ManhattanLife] SQL mapping returned 0 rows — skipping.")
            return out_df

    # Merge mapping
    src_key = raw_df[plan_code_col].astype(str).str.strip()
    map_df = map_df.drop_duplicates(subset=["PlanCode"]).copy()
    map_df.index = map_df["PlanCode"].astype(str).str.strip()

    mapped_policy = src_key.map(map_df["PolicyNumber"]).fillna("")
    mapped_name = src_key.map(map_df["ProductName"]).fillna("")

    if "ProductType" not in out_df.columns:
        out_df["ProductType"] = ""
    if "PlanName" not in out_df.columns:
        out_df["PlanName"] = ""

    out_df["ProductType"] = mapped_policy.where(mapped_policy.ne(""), out_df["ProductType"])
    out_df["PlanName"] = mapped_name.where(mapped_name.ne(""), out_df["PlanName"])

    print(f"[ManhattanLife] Populated ProductType/PlanName for {(mapped_policy!='').sum()} rows.")
    return out_df


In [ ]:
# ======================================================
# Cell 2: processor.py (simplified demo for testing)
# ======================================================

import pandas as pd
import os
from manhattan_mapping import is_manhattan_issuer, apply_manhattan_mapping

def log(msg: str):
    print(msg)

def run_llm_pipeline(
    *, issuer: str, paycode: str, trandate: str,
    csv_path: str, template_dir: str
) -> pd.DataFrame:
    """
    Simulated version of your full processor — runs the LLM flow,
    then calls DB enrichment for Manhattan Life.
    """
    log(f"Starting LLM pipeline | issuer={issuer} | csv={csv_path}")

    # Simulate what your LLM would produce
    df = pd.read_csv(csv_path, dtype=str).fillna("")
    out_df = df.copy()
    out_df["ProductType"] = "LLM_ProductType"
    out_df["PlanName"] = "LLM_PlanName"
    out_df["Issuer"] = issuer
    out_df["TranDate"] = trandate
    out_df["PayCode"] = paycode

    log(f"[LLM] Finished transformation → {len(out_df)} rows")

    # === Manhattan Life DB hook ===
    if is_manhattan_issuer(issuer):
        log("[INFO] Manhattan Life detected. Applying DB enrichment…")
        try:
            out_df = apply_manhattan_mapping(
                out_df=out_df,
                raw_df=df,
                plan_code_col="PlanCode",
                load_task_id=int(os.getenv("MANHATTAN_LOAD_TASK_ID", "13449")),
                company_issuer_id=int(os.getenv("MANHATTAN_COMPANY_ISSUER_ID", "2204")),
                server=os.getenv("SQL_SERVER", "QWVIDBSQLB401.ngquotit.com"),
                database=os.getenv("SQL_DATABASE", "NGCS"),
                driver=os.getenv("SQL_DRIVER", "ODBC Driver 18 for SQL Server"),
                encrypt=os.getenv("SQL_ENCRYPT", "no").lower() in ("1","true","yes","y"),
                trust_server_certificate=os.getenv("SQL_TRUST_SERVER_CERT", "yes").lower() in ("1","true","yes","y"),
            )
            log("[INFO] Manhattan Life enrichment complete.")
        except Exception as e:
            log(f"[WARN] Manhattan Life mapping failed: {e}")
    else:
        log("[INFO] Non-Manhattan issuer — skipping DB mapping.")

    return out_df


In [ ]:
# ======================================================
# Cell 3: Combine and test
# ======================================================

import pandas as pd
import os

# --- Configuration ---
os.environ["SQL_SERVER"] = "YOUR_SQL_SERVER"
os.environ["SQL_DATABASE"] = "YOUR_DATABASE"
os.environ["MANHATTAN_LOAD_TASK_ID"] = "13449"
os.environ["MANHATTAN_COMPANY_ISSUER_ID"] = "2204"

# --- Create a dummy input CSV ---
csv_path = "test_manhattan.csv"
sample = pd.DataFrame({
    "PlanCode": ["ML1001", "ML2002", "ML3003"],
    "OtherField": ["A", "B", "C"]
})
sample.to_csv(csv_path, index=False)

# --- Run the processor ---
issuer = "Manhattan Life"
final_df = run_llm_pipeline(
    issuer=issuer,
    paycode="P01",
    trandate="10/30/2025",
    csv_path=csv_path,
    template_dir="./"
)

print("\n✅ Final Output Preview:")
display(final_df.head())
